# Solving an IVP
We consider an IVP of the form $y'' = g(t)-\alpha y$ with $y(0) = v_0$ and $y'(0) = v'_0$. The independent variable is time ($t$) and we want to discretize to avoid working with a continuum amount of values. Hence, we set a time quantum $\varepsilon$ to be a small time interval and consider the equation on values that are multiples of the quantum. If the quantum is small enough, the following equations will be a good approximation of the problem (they can be derived from the derivative and second derivative definitions).
$$ \begin{align*}
       y'(k\varepsilon) &= \frac{y(k\varepsilon) - y((k-1)\varepsilon)}{\varepsilon} &&\mbox{(backward difference)}\\
       y''(k\varepsilon) &= \frac{y((k+1)\varepsilon) - 2y(k\varepsilon) + y((k-1)\varepsilon)}{\varepsilon^2} &&\mbox{(center difference)}
\end{align*} $$

For simplicity, we will write $y_{k} := y(k\varepsilon)$, we then rewrite the second equation. $$y''(k\varepsilon) = \frac{y_{k+1} - 2y_{k} + y_{k-1}}{\varepsilon^2}$$

If we use this in our differential equation, we get a recurrence formula for $y_k$.
$$ \begin{align*}
y''(k\varepsilon) = \frac{y_{k+1} - 2y_{k} + y_{k-1}}{\varepsilon^2} &= g(k\varepsilon) - \alpha y_k\\
y_{k+1} - 2y_{k} + y_{k-1} &= \varepsilon^2(g(k\varepsilon) - \alpha y_k)\\
y_{k+1} &= (2-\alpha\varepsilon^2)y_k -y_{k-1} + \varepsilon^2\cdot g(k\varepsilon)
\end{align*} $$

The initial values give us the start of this problem, in our case, we have $y_0 = v_0$ and $y_1 = v_0 + v'_0\varepsilon$. Now, we can solve the recurrence using forward substitution.

The following code implements this IVP solver. The specific IVP we are solving has $\alpha = \frac{g}{\ell}$ and $g(t) = \frac{1}{2}\mu\cdot S(t) + v^2(t) \cdot Cx + \frac{1}{m\cdot \ell}$ for some functions $S$ and $v^2$. It is modelling the motion of a cable car on its cable with known wind behaviour. The original formula was the following.
$$ \ddot{\theta} + \frac{g}{\ell}\theta = \frac{1}{2}\mu\cdot S(t) + v^2(t) \cdot Cx + \frac{1}{m\cdot \ell} $$

In [25]:
## - Modules - ##
import numpy as np
import matplotlib.pyplot as plt

## - Constants - ##
t_end = 1 # ending time of the simulation (t_start = 0)
steps = 100
domain = np.linspace(0,t_end,steps)
eps = t_end/steps # value of epsilon (the time quantum)
epsq = eps*eps # epsilon^2
teta0=17
petitomega=2
omega0=20
grandomega=5
m=0.225
L=0.17
J=5
alpha = -1*pow(omega0,2)

## - Defining functions - ##
def g1(t):
    return np.cos(petitomega*t)*teta0*(petitomega**2-omega0**2)
    
def g2(t):
    return m*(pow(L,2))*(pow(petitomega,3))*(pow(teta0,3))*np.sin(petitomega*t)/J
    
def g3(t):
    return (1/(grandomega**2-4*petitomega**2))*2*petitomega*(np.cos(grandomega*t)-np.cos(2*petitomega*t))
    
def g4(t):
    return (1/(grandomega**2-petitomega**2))*np.cos(petitomega*t)*(grandomega*np.sin(grandomega*t)-petitomega*np.sin(petitomega*t))
def g(t):
    # This is the function g(t)*epsilon^2
    return g1(t)+g2(t)*(g3(t)+g4(t))

g_values = list(map(g, domain)) # these are the values of eps^2 * g(t) that we will need

def recSolver(a,b,constants, v_0, v_1):
    # solves a recurrence of the type y_(k+1) = ay_k + b_(k-1) + c_k with y_0 = v_0 and y_1 = v_1
    values = [v_0, v_1] # initial values
    for i in range(2, len(constants)): # iterate from y_2 to y_n, n is the last constant c_n we have
        y_next = values[-1]*a + values[-2]*b + epsq*constants[i-1] #computes next value
        values.append(y_next) # adds it to the list
    return values # return the list of values we found

solution = recSolver(2-alpha*epsq, -1,g_values, 0.1, -0.005)
#plotting the solution
plt.plot(domain, list(map(float,solution)))
plt.show()

[ 0.          0.01010101  0.02020202  0.03030303  0.04040404  0.05050505
  0.06060606  0.07070707  0.08080808  0.09090909  0.1010101   0.11111111
  0.12121212  0.13131313  0.14141414  0.15151515  0.16161616  0.17171717
  0.18181818  0.19191919  0.2020202   0.21212121  0.22222222  0.23232323
  0.24242424  0.25252525  0.26262626  0.27272727  0.28282828  0.29292929
  0.3030303   0.31313131  0.32323232  0.33333333  0.34343434  0.35353535
  0.36363636  0.37373737  0.38383838  0.39393939  0.4040404   0.41414141
  0.42424242  0.43434343  0.44444444  0.45454545  0.46464646  0.47474747
  0.48484848  0.49494949  0.50505051  0.51515152  0.52525253  0.53535354
  0.54545455  0.55555556  0.56565657  0.57575758  0.58585859  0.5959596
  0.60606061  0.61616162  0.62626263  0.63636364  0.64646465  0.65656566
  0.66666667  0.67676768  0.68686869  0.6969697   0.70707071  0.71717172
  0.72727273  0.73737374  0.74747475  0.75757576  0.76767677  0.77777778
  0.78787879  0.7979798   0.80808081  0.81818182  0.

C:\Program Files\Anaconda3\lib\site-packages\matplotlib\ticker.py:1424: RuntimeWarning: invalid value encountered in double_scalars
  extra_bins = int(divmod((best_vmax - vmax), step)[0])


ValueError: cannot convert float NaN to integer